In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [4]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
print(len(y))
print(len(y[y > 0]))

250000
85667


In [30]:
import pandas as pd
from IPython.display import display

hb = pd.read_csv(DATA_TRAIN_PATH, sep=',')
pd.options.display.max_columns = None
#hb = hb.drop(['Id', 'Prediction'], 1) # Not in tX
# Remove those who have negative DER_mass_MMC
hb = hb[hb.DER_mass_MMC > -999.0]

'''
print(hb.columns.get_loc("DER_deltaeta_jet_jet"))
print(hb.columns.get_loc("DER_mass_jet_jet"))
print(hb.columns.get_loc("DER_prodeta_jet_jet"))
print(hb.columns.get_loc("DER_lep_eta_centrality"))
print(hb.columns.get_loc("PRI_jet_leading_pt"))
print(hb.columns.get_loc("PRI_jet_leading_eta"))
print(hb.columns.get_loc("PRI_jet_leading_phi"))
print(hb.columns.get_loc("PRI_jet_subleading_pt"))
print(hb.columns.get_loc("PRI_jet_subleading_eta"))
print(hb.columns.get_loc("PRI_jet_subleading_phi"))
'''
# TEST -> remove all negative DER_deltaeta_jet_jet
hb = hb[hb.DER_deltaeta_jet_jet > -999.0]
# We could also remove the annoying columns (10 columns less)
#hb = hb.drop(['DER_deltaeta_jet_jet', 'DER_mass_jet_jet', 'DER_prodeta_jet_jet', 'DER_lep_eta_centrality', 'PRI_jet_leading_pt', 'PRI_jet_leading_eta', 'PRI_jet_leading_phi', 'PRI_jet_subleading_pt', 'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi'], 1)

#print(f"DER_mass_MMC has {hb[hb.DER_mass_MMC >= 0.0].shape[0]} non negative values.")
#print(f"DER_mass_MMC has {hb[hb.DER_mass_MMC > 0.0].shape[0]} positive values.")

#print(f"DER_deltaeta_jet_jet has {hb[hb.DER_deltaeta_jet_jet >= 0.0].shape[0]} non negative values.")
#print(f"DER_deltaeta_jet_jet has {hb[hb.DER_deltaeta_jet_jet > 0.0].shape[0]} positive values.")

#print(f"DER_mass_jet_jet has {hb[hb.DER_mass_jet_jet >= 0.0].shape[0]} non negative values.")
#print(f"DER_mass_jet_jet has {hb[hb.DER_mass_jet_jet > 0.0].shape[0]} positive values.")


ids = hb.Id.to_numpy()
yb = hb.Prediction.to_numpy()
y = np.ones(len(yb))
y[np.where(yb=='b')] = -1

hb = hb.drop(['Id', 'Prediction'], 1)
hb = (hb - hb.mean()) / hb.std()
tX = hb.to_numpy()

In [263]:
#hb.head()
#hb.describe()

68114
31894


## Do your thing crazy machine learning thing here :) ...

In [6]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    """
    Linear regression using gradient descent.
    Uses MSE.
    
    Parameters
    ----------
    y:  ndarray
        the labels
    tx: ndarray
        vector x tilde, i.e. the parameters with a bias term
    initial_w: ndarray
        initial weight vector
    max_iters: int
        maximum number of iterations
    gamma: float
        learning rate

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """
    
    def compute_e(y, tx, w):
        return y - tx @ w
    
    def compute_loss(n2, e):
        return (e.T @ e) / n2
    
    def compute_gradient(tx, n, e):
        return - tx.T @ e / n
    
    loss = 0
    w = initial_w
    n = y.shape[0]
    n2 = n*2
    
    for n_iter in range(max_iters):
        e = compute_e(y, tx, w)
        gradient = compute_gradient(tx, n, e)
        loss = compute_loss(n2, e)
        
        # Update weights
        w -= gamma * gradient
        #print("Gradient Descent({bi}/{ti}): loss={l}, w={w}".format(
        #      bi=n_iter, ti=max_iters - 1, l=loss, w=w[0]))

    return w, loss
'''
weights = np.array([])
for i in range(100):
    initial_w = np.full(tX.shape[1], i/100)
    max_iters = 100
    gamma = 0.3
    w, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)
    weights = np.append(weights, loss)
idx = np.argmin(weights)
'''
initial_w = np.zeros(tX.shape[1])
max_iters = 1000
gamma = 0.3
w, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)
print(loss)

0.4076666745664875


In [31]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    """
    Linear regression using gradient descent.
    Uses MAE.
    
    Parameters
    ----------
    y:  ndarray
        the labels
    tx: ndarray
        vector x tilde, i.e. the parameters with a bias term
    initial_w: ndarray
        initial weight vector
    max_iters: int
        maximum number of iterations
    gamma: float
        learning rate

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """
    
    def compute_e(y, tx, w):
        return y - tx @ w
    
    def compute_loss(n, e):
        return 1/n * np.sum(np.abs(e))
    
    def compute_gradient(tx, n, e):
        e = y - tx @ w
    
        return -1/n*tx.T @ np.sign(e)
    
    loss = 0
    w = initial_w
    n = y.shape[0]
    n2 = n*2
    
    for n_iter in range(max_iters):
        e = compute_e(y, tx, w)
        gradient = compute_gradient(tx, n, e)
        loss = compute_loss(n2, e)
        
        # Update weights
        w -= gamma * gradient
        #print("Gradient Descent({bi}/{ti}): loss={l}, w={w}".format(
        #      bi=n_iter, ti=max_iters - 1, l=loss, w=w[0]))

    return w, loss

initial_w = np.zeros(tX.shape[1])
max_iters = 1000
gamma = 0.3
w, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)
print(loss)

0.3623498963285233


In [254]:
# TODO
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    """
    Linear regression using stochastic gradient descent.
    Uses MAE.
    
    Parameters
    ----------
    y:  ndarray
        the labels
    tx: ndarray
        vector x tilde, i.e. the parameters with a bias term
    initial_w: ndarray
        initial weight vector
    max_iters: int
        maximum number of iterations
    gamma: float
        learning rate

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """
    
    def compute_e(y, tx, w):
        return y - tx @ w
    
    def compute_loss(n2, e):
        return (e.T @ e) / n2
    
    def compute_gradient(tx, n, e):
        return - tx.T @ e / n
    
    loss = 0
    w = initial_w
    n = y.shape[0]
    n2 = n*2
    data_size = len(y)
    shuffled_indices = np.random.permutation(np.arange(data_size))
    shuffled_y = y[shuffled_indices]
    shuffled_tx = tx[shuffled_indices]
    for n_iter, by, btx in zip(range(max_iters), shuffled_y, shuffled_tx):
        e = compute_e(by, btx, w)
        gradient = compute_gradient(btx, n, e)
        loss = compute_loss(n2, e)
        
        # Update weights
        w -= gamma * gradient
        print("Gradient Descent({bi}/{ti}): loss={l}, w={w}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w=w[0]))

    return w, loss

initial_w = np.full(tX.shape[1], 0.1)
max_iters = 100
gamma = 1.2e-5
w, loss = least_squares_SGD(y, tX, initial_w, max_iters, gamma)
print(loss)

ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [40]:
# TODO
def least_squares(y, tx):
    """
    Linear regression using normal equations.
    Use MSE loss function
    
    Parameters
    ----------
    y:  ndarray
        the labels
    tx: ndarray
        vector x tilde, i.e. the parameters with a bias term

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """    
    def compute_e(y, tx, w):
        return y - tx @ w
    
    def compute_loss(n2, e):
        return (e.T @ e) / n2
    
    w = np.linalg.solve(tx.T @ tx, tx.T @ y)
    
    return w, compute_loss(y.shape[0]*2, compute_e(y, tx, w))

w, loss = least_squares(y, tX)
print(loss)
print(w)

0.3698965364221289
[-5.32998865e-02 -1.26587150e-01 -1.11363285e-01  2.43813778e-01
 -1.22515002e-02  2.06831545e-01  1.19017540e-02  2.85870869e-01
 -6.22024253e-02 -3.20799044e+02 -1.90769492e-01  1.13537522e-01
  1.55847156e-01  7.32390150e+01 -4.60076213e-03  4.68989539e-04
  7.11501582e+01 -6.18816779e-03  3.77252159e-03  5.37365302e-02
  5.63666694e-03 -1.25425938e-01 -9.06055686e-02 -2.03041643e-01
  5.72629002e-03  6.38749556e-03  1.36129488e-02  5.16397394e-03
 -3.12506068e-03  2.66851042e+02]


In [ ]:
# TODO
def ridge_regression(y, tx, lambda_):
    """
    Ridge regression using normal equations.
    
    Parameters
    ----------
    y : ndarray
        Description of y
    ...

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """    
    None

In [ ]:
# TODO
def logistic_regression(y, tx, lambda_):
    """
    Logistic regression using gradient descent or SGD.
    
    Parameters
    ----------
    y : ndarray
        Description of y
    ...

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """    
    None

In [ ]:
# TODO
def reg_logistic_regression(y, tx, lambda_):
    """
    Regularized logistic regression using gradient descent or SGD.
    
    Parameters
    ----------
    y : ndarray
        Description of y
    ...

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """    
    None

## Generate predictions and save ouput in csv format for submission:

In [20]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [41]:
OUTPUT_PATH = 'predictions.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(w, tX_test)#[:, [0, 1, 2, 3, 4, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]]) # Selected desired columns
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [42]:
print(len(y_pred))
print(len(y_pred[y_pred > 0]))

568238
0
